In [1]:
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, TrainingArguments)
from trl import SFTTrainer


model_path = "../models/llama3-ko-alwayssaewoo_problem-adapter"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [2]:
quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = quant_config
).to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
     model_path,
    device_map="cuda",
    dtype="auto"
).to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [5]:
training_args = TrainingArguments(
    output_dir = "./arg", # 체크포인트, 로그 저장 경로
    save_total_limit = 2, # ./arg 저장 체크포인트 최대 개수
        logging_steps = 10, # 로그 출력 간격
        num_train_epochs = 3, # 에폭
        per_device_train_batch_size = 4, # GPU당 배치 크기
        gradient_accumulation_steps = 8, # gradient accumulation (미니 배치 후 업데이트)
        learning_rate = 2e-4, # 학습률
        save_steps = 50, # 모델 저장 업데이트 간격
        fp16 = True, # fp16 설정
        eval_strategy = "no" # 모델 평가 no
    )

In [8]:
data_path = "../problem_data/train.jsonl"

def combine(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output_text = example["output"]
    example["text"] = f"<s>[INST] {instruction} {input_text} [/INST] {output_text}</s>"
    return example

dataset = load_dataset("json", data_files=data_path, split="train")
dataset = dataset.map(combine)
test_data_path = "../data/test.jsonl"
test_dataset = load_dataset("json", data_files=test_data_path, split="train")
test_dataset = test_dataset.map(combine)

In [11]:
# LoRA 세팅
lora_config = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 16,
    bias = "none",
    task_type = "CAUSAL_LM"
) 

In [12]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=test_dataset,  
    peft_config=lora_config,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to eval dataset:   0%|          | 0/103 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/103 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/103 [00:00<?, ? examples/s]

In [13]:
eval_result = trainer.evaluate()
print(eval_result)

{'eval_loss': 2.1008527278900146, 'eval_model_preparation_time': 0.0281, 'eval_runtime': 6.7179, 'eval_samples_per_second': 15.332, 'eval_steps_per_second': 1.935, 'eval_entropy': 1.3970987705083995, 'eval_num_tokens': 0.0, 'eval_mean_token_accuracy': 0.5705152383217444}


In [17]:
from transformers import pipeline

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

for i in range(1):
    prompt = "주어진 주제를 기반으로 주관식 문제를 생성하라."#test_dataset[i]["instruction"]
    print("=== 입력 ===")
    print(prompt)
    print("=== 모델 출력 ===")
    print(gen(prompt, max_new_tokens=200)[0]["generated_text"])
    print()

Device set to use cuda


=== 입력 ===
주어진 주제를 기반으로 주관식 문제를 생성하라.
=== 모델 출력 ===
주어진 주제를 기반으로 주관식 문제를 생성하라. 주제는 "가족"이다. 주관식 문제는 다음과 같은 형태로 작성된다: "가족은___."으로 시작한다. 주관식 문제는 답변자의 개인적인 생각과 감정을 반영하므로, 여러 사람에게 다르게 답할 수 있다. 주제에 대한 주관식 문제를 하나씩 작성해보자.

1. 가족은___.
2. 가족은___.
3. 가족은___.
4. 가족은___.
5. 가족은___.

1. 가족은 나의 삶에 불가결한 존재입니다.
2. 가족은 내 인생에서 가장 소중한 재산입니다.
3. 가족은 내 삶의 안정과 행복의 근본입니다.
4. 가족은 내 인생에서 가장 큰 사랑과 희생을 주는 존재입니다.
5. 가족은 내 인생의 가장 큰 보상입니다. 

이러한 주관식 문제는 다양한 답변을 통해 다양한

